In [2]:
import pandas as pd
import numpy as np
import re

In [156]:
class BERT():
    
    def __init__(self):
        self.candidates = ['Trump', 'Bernie', 'Sanders', 'Biden', 'Warren', 'Buttigieg', 'Bloomberg', 
                           'Klobuchar', 'Yang', 'Steyer', 'Gabbard']
        self._data = pd.read_csv('data/sentence_data.csv')[['article_text']]

    def data(self):
        data = self._data.copy()
        
        # add labels
        data['label'] = data['article_text'].str.extract('({})'.format('|'.join(self.candidates)), 
                            flags = re.IGNORECASE, expand = False).str.lower().fillna('')
        data['label'] = np.where(data['label'].str.contains('bernie'), 'sanders', data['label'])

        # remove candidate names
        data['article_text'] = data['article_text'].str.replace('Donald', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Trump', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Bernie', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Sanders', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Joe', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Biden', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Elizabeth', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Warren', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Andrew', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Yang', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Tom', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Steyer', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Tulsi', '[candidate]')
        data['article_text'] = data['article_text'].str.replace('Gabbard', '[candidate]')

        self._data = data
        
        return self._data

In [157]:
self = BERT()